In [ ]:
# Find the maximum value in warped_image
max_value = np.max(warped_image)

# Find the coordinates of the maximum value
max_coords = np.argwhere(warped_image == max_value)

print("Maximum value:", max_value)
print("Coordinates of maximum value:", max_coords)

In [ ]:
"""Warp image using thin-plate spline transformation."""
image = all_data_dict[os.path.basename(afm_file)]['afm_img']

h, w = image.shape[:2]


# Create a grid of pixel coordinates
grid_x, grid_y = np.meshgrid(np.arange(w), np.arange(h))
grid_points = np.column_stack([grid_x.ravel(), grid_y.ravel()])

rbf_x = all_data_dict[os.path.basename(afm_file)]['rbf_x']
rbf_y = all_data_dict[os.path.basename(afm_file)]['rbf_y']

# Compute the transformed coordinates
warped_x = rbf_x(grid_points).reshape(h, w).astype(np.float32)
warped_y = rbf_y(grid_points).reshape(h, w).astype(np.float32)

# Remap the image
warped_image = cv2.remap(image, warped_x, warped_y, interpolation=cv2.INTER_CUBIC)

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(image, cmap='afmhot')
ax[0].set_title('Original AFM Image')
ax[1].imshow(warped_image, cmap='afmhot')
ax[1].set_title('Warped AFM Image')


In [ ]:
cx, cy = (312.45454545454544, 44.81818181818182)

def load_feature_points(csv_path):
    """Load feature points from CSV file."""
    df = pd.read_csv(csv_path)
    source_points = df[['x2', 'y2']].values  # Feature positions in image 1
    target_points = df[['x1', 'y1']].values  # Corresponding positions in image 2
    return source_points, target_points


# Use rbf_x and rbf_y to transform the coordinates
blank_array = np.zeros((512, 512))
blank_array[int(cy), int(cx)] = 255

source_points, target_points = load_feature_points(all_data_dict['fat_end_outside_L_03.0_00000.png']['alignment_file'])

"""Warp image using thin-plate spline transformation."""
h, w = blank_array.shape[:2]

# Scale the source points
source_points_scaled = source_points

# Create a grid of pixel coordinates
grid_x, grid_y = np.meshgrid(np.arange(w), np.arange(h))
grid_points = np.column_stack([grid_x.ravel(), grid_y.ravel()])

# Interpolate transformation using RBF (thin-plate spline kernel)
rbf_x = RBFInterpolator(source_points_scaled, target_points[:, 0], kernel='thin_plate_spline')
rbf_y = RBFInterpolator(source_points_scaled, target_points[:, 1], kernel='thin_plate_spline')

# Compute the transformed coordinates
warped_x = rbf_x(grid_points).reshape(h, w).astype(np.float32)
warped_y = rbf_y(grid_points).reshape(h, w).astype(np.float32)

# Remap the image
warped_image = cv2.remap(blank_array, warped_x, warped_y, interpolation=cv2.INTER_CUBIC)



warped_image_afm = cv2.remap(all_data_dict['fat_end_outside_L_03.0_00000.png']['afm_img'], warped_x, warped_y, interpolation=cv2.INTER_CUBIC)

# Find the maximum value in warped_image
max_value = np.max(warped_image)
# Find the coordinates of the maximum value
max_coords = np.argwhere(warped_image == max_value)
cl_cy, cl_cx = max_coords[0]  # Get the first coordinate pair

print(cl_cx, cl_cy)

cl_cx = rbf_x(np.array([[cx, cy]]))[0]
cl_cy = rbf_y(np.array([[cx, cy]]))[0] + 256

print(cl_cx, cl_cy)

# Plot the AFM image with the random spot highlighted
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Plot the AFM image
axes[0].imshow(all_data_dict['fat_end_outside_L_03.0_00000.png']['afm_img'], cmap='afmhot')
afm_circle = plt.Circle((cx, cy), radius=10, color='red', fill=False, linewidth=2)
axes[0].add_patch(afm_circle)
# axes[0].imshow(blank_array, cmap='gray', alpha=0.5)  # Overlay the blank array
axes[0].set_title("AFM Image with Random Spot Highlighted", fontsize=10, y=-0.05)
axes[0].axis('off')

# ax4 = plt.subplot(2, 2, 4)
# ax4.imshow(image2, cmap='viridis')
# ax4.imshow(warped_image1, cmap='afmhot', alpha=0.3)  # Overlay with transparency
# ax4.set_title("Overlay of Aligned AFM Image on CL Image")
# ax4.set_xlim(0, 256)
# ax4.set_ylim(256, 0)  # Invert y-axis to match image coordinates
# add_scale_bar(ax4, 1, 4)  # 1 µm scale bar for 4 µm image

# Plot the CL image
axes[1].imshow(all_data_dict['fat_end_outside_L_03.0_00000.png']['cl_img'], cmap='viridis')
#cl_circle = plt.Circle((cl_cx, cl_cy), radius=7, color='blue', fill=False, linewidth=2)
cl_square = plt.Rectangle((cl_cx - 7.5, cl_cy - 7.5), 15, 15, color='blue', fill=False, linewidth=2)
axes[1].add_patch(cl_square)
axes[1].imshow(warped_image_afm, cmap='gray', alpha=0.5)  # Overlay the warped image
axes[1].set_title("CL Image with Corresponding Cut-Out for ML Training", fontsize=10, y=-0.05)
axes[1].set_xlim(0, 256)
axes[1].set_ylim(256, 0)
axes[1].axis('off')

plt.tight_layout()
plt.show()